In [1]:
# syft absolute
import syft as sy
from syft.serde.recursive import TYPE_BANK
from syft.service.log.log import SyftLog
from syft.types.syft_object_registry import SyftObjectRegistry

from syft.protocol.data_protocol import get_data_protocol

In [2]:
dp = get_data_protocol()

In [3]:
# dp.protocol_history.keys()

In [4]:
node = sy.orchestra.launch(
    name="test_upgradbility",
    dev_mode=True,
    local_db=True,
    n_consumers=2,
    create_producer=True,
)


Staging Protocol Changes...
Object in Document Store that needs migration:  [<class 'syft.service.settings.settings.NodeSettings'>, <class 'syft.service.log.log.SyftLog'>, <class 'syft.service.job.job_stash.Job'>, <class 'syft.service.network.node_peer.NodePeer'>, <class 'syft.service.sync.sync_state.SyncState'>, <class 'syft.service.output.output_service.ExecutionOutput'>]
Migrating data for: NodeSettings table.
Migrating data for: SyftLog table.
Migrating data for: JobItem table.
Migrating data for: NodePeer table.
Migrating data for: SyncState table.
Migrating data for: ExecutionOutput table.
could not find ExecutionOutput 1 in ObjectRegistry
('ExecutionOutput', 1)


Exception: Failed to migrate data for ExecutionOutput. Error: cannot unpack non-iterable NoneType object

In [ ]:
%debug

In [ ]:
# %debug

## Appendix

In [ ]:
log = SyftLog(job_id=sy.UID())

eventually there are 3 cases
- its s SyftObject with canonical_name and version (this works)
- its not a syftobject, it still has a canonical_name and version (currently this is handled by splitting the fqn) (requires refactor)

- it has a fqn (can either be SyftObject or non SyftObject) from previous version
  - For deserialization of SyftObjects, this will give you the latest object
 

  

In [ ]:

# we want to know, for this fqn, what is the object version. It should always be the one described in 4, because thats 
# the only migration we support
# new_version = any([v["action"] == "add" for v in dp.protocol_history["dev"]["object_versions"]["SyftLog"].values()])
 # has the mapping of all the protocol version and classes version that are added or removed.

In [ ]:
# SyftLog.__canonical_name__

In [ ]:
# version_mutations

In [ ]:
# bts = sy.serialize(log, to_bytes=True)

In [ ]:
# with open("log086", "rb") as f:
#     bts = f.read()

In [ ]:
log_des = sy.deserialize(bts, from_bytes=True)

SyftLog 3
3


In [ ]:
log_des

```python
class SyftLogV3:
  id: str = a0adb0f277f54935acf25dc966a1de77
  stdout: str = "ABC"
  stderr: str = "DEF"

```

if we dont have an old version, how are we going to test?

- release a beta and test from there? (hard to test during development)
- from a branch? (how does this work with protocol versions?)
- if there is a fqn, we assume that its 0.8.6?
- 

In [ ]:

TYPE_BANK["syft.service.log.log.SyftLog"]

In [ ]:
#how do we test, because the old version does not contain any information

In [ ]:
from syft.types.syft_object import SyftObjectRegistry

In [ ]:
SyftObjectRegistry.__object_version_registry__["SyftLog_3"]

currently `fqn` -> all serialization properties

this is problematic because `fqn` can point to different object versions (or to nothing) in different versions

instead we want `version_string` to point to all those things


we could either
- remove TYPE_BANK
- make `version_string` point to `fqn` which and get the serialization properties directly



In [ ]:
def get_obj_registry_str_from_fqn(fqn):
    (nonrecursive,
    serialize,
    deserialize,
    attribute_list,
    exclude_attrs_list,
    serde_overrides,
    hash_exclude_attrs,
    cls,
    attribute_types,
    version) = TYPE_BANK[fqn]
    object_registry_str = f"{cls.__canonical_name__}_{cls.__version__}"

    return object_registry_str
        

In [ ]:
name = get_obj_registry_str_from_fqn("syft.service.log.log.SyftLog")

In [ ]:
SyftObjectRegistry.__object_version_registry__[name]

In [ ]:

# for x in SyftObjectRegistry.__object_version_registry__.keys():
#     print(x)

In [ ]:
sy.__version__

In [ ]:
with open("log086", "rb") as f:
    bts = f.read()

In [ ]:
log = sy.deserialize(bts, from_bytes=True)

ValidationError: 1 validation error for SyftLog
job_id
  Field required [type=missing, input_value={'id': <UID: a0adb0f277f5...ft_node_location': None}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.6/v/missing

In [ ]:
%debug

> /opt/anaconda3/envs/syft/lib/python3.12/site-packages/pydantic/main.py(171)__init__()
    169         # `__tracebackhide__` tells pytest and some other tools to omit this function from tracebacks
    170         __tracebackhide__ = True
--> 171         self.__pydantic_validator__.validate_python(data, self_instance=self)
    172 
    173     # The following line sets a flag that we use to determine when `__init__` gets overridden by the user



ipdb>  u


> /Users/koen/workspace/PySyft/packages/syft/src/syft/types/syft_object.py(590)__init__()
    588 
    589     def __init__(self, **kwargs: Any) -> None:
--> 590         super().__init__(**kwargs)
    591         self._syft_set_validate_private_attrs_(**kwargs)
    592         self.__post_init__()



ipdb>  u


> /Users/koen/workspace/PySyft/packages/syft/src/syft/serde/recursive.py(382)rs_proto2object()
    380                 setattr(obj, attr_name, attr_value)
    381         else:
--> 382             obj = class_type(**kwargs)
    383 
    384     else:



...

ipdb>  class_type


<class 'syft.service.log.log.SyftLog'>


ipdb>  index_syft_by_module_name(proto.fullyQualifiedName)


<class 'syft.service.log.log.SyftLog'>


ipdb>  q


In [ ]:
import syft as syft

In [ ]:
import sys

In [ ]:
from syft.serde.recursive import TYPE_BANK

In [ ]:
# TYPE_BANK["syft.service.log.log.SyftLog"]

In [ ]:
from typing import Type

In [ ]:
class SyftClassRegistry():
    __class_registry__: dict[tuple, Type] = {}
    is_built = False

    @classmethod
    def build():
        for path, v in TYPE_BANK:
            (nonrecursive,
            serialize,
            deserialize,
            attribute_list,
            exclude_attrs_list,
            serde_overrides,
            hash_exclude_attrs,
            cls,
            attribute_types,
            version) = v
            cls.__class_registry__[cls.__canonical_name__, cls.__version__] = cls
        cls.is_build=True
            
    @classmethod
    def get(cls, cannonical_name: str, version: str):
        if not cls.is_built:
            cls.build()
        return cls.__class_registry__[cannonical_name, version]
        
            
            




In [ ]:
(nonrecursive,
serialize,
deserialize,
attribute_list,
exclude_attrs_list,
serde_overrides,
hash_exclude_attrs,
cls,
attribute_types,
version) = TYPE_BANK["syft.service.settings.settings.NodeSettingsV2"]

In [ ]:
cls.__canonical_name__

'SyftLog'

In [ ]:
cls

syft.service.log.log.SyftLog

In [ ]:
(nonrecursive,
serialize,
deserialize,
attribute_list,
exclude_attrs_list,
serde_overrides,
hash_exclude_attrs,
cls,
attribute_types,
version) = TYPE_BANK["syft.service.log.log.SyftLog"]

In [ ]:
from syft.protocol.data_protocol import get_data_protocol

dp = get_data_protocol()

In [ ]:
# dp.current

In [ ]:
# dp.protocol_history

In [ ]:
version = "3"

In [ ]:
sys.modules["syft.service.log.log"].SyftLogV1

AttributeError: module 'syft.service.log.log' has no attribute 'SyftLogV1'

- problem 1: it indexes classes by their location in the previous version, which may change in the current version
- problem 2: resolving the name of the class to the version is dynamic and has a different meaning accross versions, therefore having just the class name is not enough

Solution: create a `SyftClassRegistry` that follows the same patterns as the `stage_` logic


In [ ]:
syft.service.log.log

<module 'syft.service.log.log' from '/Users/koen/workspace/PySyft/packages/syft/src/syft/service/log/log.py'>

In [ ]:
syft.__dict__["service"]

<module 'syft.service' from '/Users/koen/workspace/PySyft/packages/syft/src/syft/service/__init__.py'>